## Creating one json file per continent to avoid reading the full csv file each time

In [1]:
from shapely.geometry import Polygon, Point
import geopandas
import csv
import numpy as np
import json
import os

region_names = ['NORTH-AMERICA']#, 'SOUTH-AMERICA', 'EUROPE', 'OCEANIA']#'AFRICA', 'ASIA']#, 
scenario = 'RCP85'
horizon = '2100'

# -- Retrieve the definitions of the subregions from the csv file of the reference regions
# -----------------------------------------------------------------
def retrieve_AR6regions_for_region(region_name):
    
    regions_filename='/home/jservon/Chapter12_IPCC/scripts/ATLAS/reference-regions/IPCC-WGI-reference-regions-v4_coordinates.csv'
    
    AR6_regions_by_continent = {
        'NORTH-AMERICA':['NWN','NEN','WNA','CNA','ENA','NCA','CAR'],
        'SOUTH-AMERICA':['NWS','NSA','SAM','NES','SWS','SES','SSA','SCA']
    }
        
    # -- Store the informations by region in the 'regions' dictionary
    regions = dict()
    with open(regions_filename) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')#, quotechar='|')
        for row in spamreader:
            if region_name in AR6_regions_by_continent:
                if row[3] in AR6_regions_by_continent[region_name]:
                    region_dict = dict()
                    lats_vect = []
                    lons_vect = []
                    tmp_polygon_vertices = []
                    for vertice in row[4:-1]:
                        if vertice:
                            dum = vertice.split('|')
                            lons_vect.append(float(dum[0]))
                            lats_vect.append(float(dum[1]))
                            tmp_polygon_vertices.append( (float(dum[0]), float(dum[1])) )
                    region_dict['polygon'] = Polygon(tmp_polygon_vertices)
                    region_dict['lons_vect'] = lons_vect
                    region_dict['lats_vect'] = lats_vect
                    #
                    regions[row[3]] = region_dict
            else:
                if region_name in row[0]:
                    region_dict = dict()
                    lats_vect = []
                    lons_vect = []
                    tmp_polygon_vertices = []
                    for vertice in row[4:-1]:
                        if vertice:
                            dum = vertice.split('|')
                            lons_vect.append(float(dum[0]))
                            lats_vect.append(float(dum[1]))
                            tmp_polygon_vertices.append( (float(dum[0]), float(dum[1])) )
                    region_dict['polygon'] = Polygon(tmp_polygon_vertices)
                    region_dict['lons_vect'] = lons_vect
                    region_dict['lats_vect'] = lats_vect
                    #
                    regions[row[3]] = region_dict
    #
    return regions


# -- Get all the subregions of the region_name and create the geopandas object
# -----------------------------------------------------------------
subregions = dict()
for region_name in region_names:
    tmp = retrieve_AR6regions_for_region(region_name)
    subregions_polygons = []
    for subregion in tmp:
        subregions_polygons.append(tmp[subregion]['polygon'])
    subregions[region_name] = geopandas.GeoSeries( subregions_polygons )
        

# -- Start a plot to check that the data is properly located in the regions
# -----------------------------------------------------------------
filename = '/home/ciles/IPCC/coastal/globalErosionProjections_Long_Term_Change_'+scenario+'_'+horizon+'.csv'

print 'Reading coastal recession data for ',scenario,horizon
with open(filename) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    res = dict()
    for region_name in region_names:
        res[region_name] = dict(
                                lons_list = [],
                                lats_list = [],
                                data_list = []
                                )
    i = 0
    for row in spamreader:
        lon = float(row[1])
        lat = float(row[0])
        # -- Check whether lon/lat is within one of the subregions
        # -- If yes, add to the plot
        for region_name in region_names:            
            if subregions[region_name].contains( Point(lon,lat) ).any():
                coastal_recession_value = float(row[5])
                res[region_name]['lons_list'].append(lon)
                res[region_name]['lats_list'].append(lat)
                res[region_name]['data_list'].append(coastal_recession_value)
                
#

Reading coastal recession data for  RCP85 2100


In [2]:
# -- Save in json
for region_name in region_names:
    outfilename = '/home/jservon/Chapter12_IPCC/data/coastal_recession/CoastalRecession_'+region_name+'_'+scenario+'_'+horizon+'.json'
    with open(outfilename, 'w') as fp:
        json.dump(res[region_name], fp, sort_keys=True, indent=4)

In [4]:
!ls -ltr /home/jservon/Chapter12_IPCC/data/coastal_recession/

total 28664
-rw-r--r-- 1 jservon ciclad 5734740 19 janv. 18:14 CoastalRecession_AFRICA_RCP85_2100.json
-rw-r--r-- 1 jservon ciclad 6953361 19 janv. 18:14 CoastalRecession_ASIA_RCP85_2100.json
-rw-r--r-- 1 jservon ciclad  157370 28 janv. 17:46 globalErosionProjections_by_AR6_region_RCP45_2050.png
-rw-r--r-- 1 jservon ciclad   12899 28 janv. 17:46 globalErosionProjections_by_AR6_region_RCP45_2050.json
-rw-r--r-- 1 jservon ciclad  157370 28 janv. 17:56 globalErosionProjections_by_AR6_region_RCP45_2100.png
-rw-r--r-- 1 jservon ciclad   12939 28 janv. 17:56 globalErosionProjections_by_AR6_region_RCP45_2100.json
-rw-r--r-- 1 jservon ciclad  157370 28 janv. 18:05 globalErosionProjections_by_AR6_region_RCP85_2050.png
-rw-r--r-- 1 jservon ciclad   12934 28 janv. 18:05 globalErosionProjections_by_AR6_region_RCP85_2050.json
-rw-r--r-- 1 jservon ciclad  157370 28 janv. 18:15 globalErosionProjections_by_AR6_region_RCP85_2100.png
-rw-r--r-- 1 jservon ciclad   12940 28 janv. 18:15 globalErosionProjec

In [8]:
res.keys()

['AFRICA', 'ASIA']

In [13]:
res['AFRICA']

{'data_list': [-236.16,
  -271.86,
  -251.81,
  -248.69,
  -109.4,
  -115.82,
  -128.33,
  -95.639,
  -68.526,
  -158.71,
  -284.96,
  -312.18,
  -272.96,
  -264.28,
  -291.84,
  -229.55,
  -240.73,
  -253.39,
  -258.6,
  -237.89,
  -214.72,
  -347.48,
  -338.25,
  -160.83,
  -253.07,
  -307.54,
  -176.26,
  -236.11,
  -125.25,
  -153.34,
  -64.175,
  -110.3,
  33.465,
  -70.723,
  -76.954,
  -190.23,
  -33.399,
  -33.399,
  -111.98,
  -59.607,
  -33.936,
  -128.19,
  -134.34,
  -96.28,
  -99.91,
  -97.471,
  -83.371,
  -160.97,
  -9.2049,
  -105.35,
  -36.073,
  -144.33,
  -111.03,
  -14.053,
  -24.372,
  -218.9,
  -644.91,
  -621.88,
  -533.86,
  -361.25,
  23.905,
  39.368,
  39.368,
  -939.57,
  -939.57,
  -284.49,
  -919.57,
  -171.7,
  -172.62,
  -923.22,
  -223.72,
  -346.44,
  -151.34,
  -151.34,
  -176.38,
  -290.22,
  -194.21,
  -178.0,
  -83.843,
  -43.323,
  1.0335,
  -97.065,
  20.078,
  -19.395,
  3.7661,
  5.1455,
  -476.03,
  -58.368,
  -431.5,
  -431.5,
  -488.53,
  -4

In [15]:
with open(outfilename) as json_file:
    tmp = json.load(json_file)
tmp.keys()

[u'lats_list', u'data_list', u'lons_list']

In [16]:
tmp['lats_list']

[-0.025684,
 -0.025028,
 -0.10571,
 -0.14486,
 -0.13893,
 -0.049534,
 -0.046747,
 -0.078968,
 -0.57905,
 -0.5823,
 -0.58709,
 -0.60737,
 -0.60994,
 -0.61253,
 -0.61477,
 -0.61749,
 -0.62021,
 -0.62411,
 -0.62729,
 -0.63423,
 -0.63557,
 -0.63316,
 -0.62883,
 -0.60783,
 -0.60337,
 -0.59892,
 -0.59365,
 -0.58854,
 -0.58427,
 -0.57925,
 -0.57513,
 -0.57102,
 -0.56691,
 -0.56233,
 -5.7273,
 -5.7295,
 -5.7324,
 -5.7365,
 -5.7407,
 -5.7444,
 -5.7635,
 -5.7661,
 -5.7696,
 -0.8324,
 -1.6842,
 -1.6881,
 -1.6916,
 -1.6953,
 -1.7962,
 -4.7511,
 -4.7467,
 -4.7423,
 -4.7378,
 -4.7075,
 -4.658,
 -4.6548,
 -4.6466,
 -4.645,
 -4.5246,
 -4.5202,
 -4.5157,
 -4.5105,
 -4.506,
 -4.5016,
 -4.4971,
 -4.4926,
 -2.5536,
 -2.5554,
 -2.5573,
 -2.5588,
 -2.5626,
 -2.5634,
 -2.564,
 -2.5644,
 -2.5644,
 -2.5646,
 -2.5674,
 -2.5694,
 -2.5707,
 -2.5714,
 -2.5702,
 -2.5691,
 -2.5715,
 -2.6417,
 -2.6459,
 -2.6509,
 -2.6607,
 -2.6639,
 -2.7402,
 -2.7457,
 -2.7613,
 -2.7665,
 -2.7697,
 -2.7779,
 -2.7834,
 -2.787,
 -2.795